In [15]:
from langchain.chat_models  import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {cuisine} food.")
    ]
)

chef_chain = chef_prompt | chat


In [16]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
        ("human", "{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and delicious recipe for Chicken Tikka Masala. 

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1/2 cup heavy cream
- 2 tsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1/2 tsp turmeric
- 1/2 tsp paprika
- Salt and pepper, to taste
- Fresh cilantro, for garnish

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, turmeric, paprika, salt, and pepper. Add the chicken pieces and mix well to coat. Marinate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and place them on a baking sheet

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1/2 cup heavy cream (you can use coconut cream for a vegan version)\n- 2 tsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1/2 tsp turmeric\n- 1/2 tsp paprika\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the tofu or paneer in the yogurt and spice mixture. Allow it to marinate for at least 1 hour.\n\n2. Instead of baking the chicken, yo